In [1]:
import numpy as np
from matplotlib import pyplot as plt
import scipy.optimize
import pickle
import sys
import os
import time
import sklearn.ensemble
import sklearn.linear_model
from importlib import reload
%matplotlib inline
reload(sklearn)

<module 'sklearn' from '/usr/local/lib/python3.4/dist-packages/sklearn/__init__.py'>

In [2]:
from sample_patches import *
from display_layer import *
from gradient import *
from autoencoder import *

In [3]:
for i in range(10):
    check_gradient()

In [6]:
hidden_size = np.array([6, 3, 6])
visible_size = 12
rho = 0.2
beta = 3.0
lambda_ = 1e-4
N = 10
data = np.random.uniform(0.1, 0.9, (N, visible_size))
theta = initialize(hidden_size, visible_size)
grad = autoencoder_loss(theta, visible_size, hidden_size, lambda_, rho, beta, data)[1]
J = lambda theta: autoencoder_loss(theta, visible_size, hidden_size, lambda_, rho, beta, data)[0]
G = compute_gradient(J, theta)
print(np.max(np.abs(G - grad)))

4.60395210289e-08


In [3]:
def gen_patches(num=10000, size=8):
    patches = np.zeros((num, size * size * 3))
    x = 0
    step = max(1, (num + 4) // 5)
    for i in range(0, num, step):
        x += 1
        f = open('./data2.7/X' + str(x) + '.pk', 'rb')
        d = pickle.load(f)
        patches[i:i + step] = sample_patches(d, step, size)
        del d
        f.close()
    return patches

In [17]:
patches = gen_patches(10000, 8)
display_layer(patches[:100], 'pathes.png')

In [6]:
hidden_size = np.array([75])
visible_size = 192
params = [{'rho': 0.06,
           'lambda_': 1e-4,
           'beta': 3.0},
          {'rho': 0.01,
           'lambda_': 1e-4,
           'beta': 3.0},
          {'rho': 0.12,
           'lambda_': 1e-4,
           'beta': 3.0},
          {'rho': 0.06,
           'lambda_': 1e-3,
           'beta': 3.0},
          {'rho': 0.06,
           'lambda_': 1e-5,
           'beta': 3.0},
          {'rho': 0.06,
           'lambda_': 1e-4,
           'beta': 6.0},
          {'rho': 0.06,
           'lambda_': 1e-4,
           'beta': 0.5},
          ]

In [7]:
for i in range(len(params)):    
    rho = params[i]['rho']
    lambda_ = params[i]['lambda_']
    beta = params[i]['beta']
    print('#{0} lambda={1:.2g} rho={2:.2g} beta={3:.2g}'.format(i, lambda_, rho, beta))
    loss = lambda theta: autoencoder_loss(theta, visible_size, hidden_size, lambda_, rho, beta, patches)
    theta = initialize(hidden_size, visible_size)
    opt_res = scipy.optimize.minimize(loss, theta, method='L-BFGS-B', jac=True, options={'disp': True,
                                                                                         'maxiter': 2000
                                                                                        })
    print(loss(theta)[0], opt_res['fun'])
    W, b = get_params(opt_res['x'], visible_size, hidden_size)
    display_layer(W[0].T, 'filters_' + str(i) + '.png')

#0 lambda=0.0001 rho=0.06 beta=3
115.28576964946446 0.2055247589413744
#1 lambda=0.0001 rho=0.01 beta=3
150.16876525275555 0.6469779141336756
#2 lambda=0.0001 rho=0.12 beta=3
87.17816049809541 0.14512721566040104
#3 lambda=0.001 rho=0.06 beta=3
114.11653950454014 0.5493606604189617
#4 lambda=1e-05 rho=0.06 beta=3
110.84767702785535 0.09518917914640546
#5 lambda=0.0001 rho=0.06 beta=6
225.88150107756556 0.20885916391694728
#6 lambda=0.0001 rho=0.06 beta=0.5
23.41827171199798 0.19989232746085617


In [ ]:
hidden_size = np.array([75])
visible_size = 192
rho = 0.06
lambda_ = 1e-4
beta = 3.0
loss = lambda theta: autoencoder_loss(theta, visible_size, hidden_size, lambda_, rho, beta, patches)
theta = initialize(hidden_size, visible_size)
opt_res = scipy.optimize.minimize(loss, theta, method='L-BFGS-B', jac=True, options={'disp': True,
                                                                                      'maxiter': 2000
                                                                                    })
theta = opt_res['x']
np.savez('theta.npz', theta=theta)

In [9]:
def run_classifiers(gen_features, test_batch_size=500):
    f = open('./data2.7/train.pk', 'rb')
    train = pickle.load(f)
    f.close()
    t1 = time.clock()
    features = gen_features(train['X'])
    t1 = time.clock() - t1
    print('Train fearutes generated. {0:.2g}s.'.format(t1))
    sys.stdout.flush()
    
    t1  = time.clock()
    rf = sklearn.ensemble.RandomForestClassifier(n_estimators=500, max_depth=30, n_jobs=3)
    rf.fit(features, train['y'].ravel())
    t1 = time.clock() - t1
    print('RF trained. {0:.2g}s.'.format(t1))
    sys.stdout.flush()
    
    t1 = time.clock()
    logr = sklearn.linear_model.LogisticRegression(solver='sag', max_iter=100)
    logr.fit(features, train['y'].ravel())
    t1 = time.clock() - t1
    print('Logistic regression trained. {0:.2g}s.'.format(t1))
    sys.stdout.flush()
    
    del features    
    del train
    
    f = open('./data2.7/test.pk', 'rb')
    test = pickle.load(f)
    f.close()
    N = test['X'].shape[0]
    rf_score = 0.0
    logr_score = 0.0
    for i in range(0, N, test_batch_size):
        print('Test batch #{0}'.format(i // test_batch_size))
        sys.stdout.flush()
        bs = min(test_batch_size, N-i)
        features = gen_features(test['X'][i:i+bs])
        y = test['y'][i:i+bs].ravel()
        rf_score += np.sum(rf.predict(features) == y)
        logr_score += np.sum(logr.predict(features) == y)
    rf_score /= N
    logr_score /= N
    print('Random forest score: {0:.4f}'.format(rf_score))
    print('Logistic regression score: {0:.4f}'.format(logr_score))    
    sys.stdout.flush()
    del features
    del test

In [33]:
def pixel_features(data):
    return data
def autoencoder_features(data, theta, hidden_size, visible_size, step=8):
    N = data.shape[0]
    D = int(math.sqrt(data.shape[1] // 3))
    layer_num = hidden_size.shape[0] // 2 + 1
    data = data.reshape(N, D, D, 3)
    features = []
    for i in range(0, D-7, step):
        for j in range(0, D-7, step):
            features.append(autoencoder_transform(theta, visible_size, hidden_size, 
                                                  layer_num, data[:, i:i+8, j:j+8, :].reshape(N, 8*8*3)))
    return np.hstack(features)

In [10]:
run_classifiers(pixel_features)

Train fearutes generated. 7e-06s.
RF trained. 8.2e+02s.
Logistic regression trained. 1.9e+03s.
Test batch #0
Test batch #1
Test batch #2
Test batch #3
Test batch #4
Test batch #5
Test batch #6
Test batch #7
Test batch #8
Test batch #9
Test batch #10
Test batch #11
Test batch #12
Test batch #13
Test batch #14
Test batch #15
Random forest score: 0.4376
Logistic regression score: 0.3401


/usr/local/lib/python3.4/dist-packages/sklearn/linear_model/sag.py:267: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [34]:
run_classifiers(lambda data: autoencoder_features(data, theta, hidden_size, visible_size, step=8))

Train fearutes generated. 9.8s.
RF trained. 1.7e+02s.


/home/timgaripov/work/prac/semestr5/05/autoencoder.py:119: RuntimeWarning: overflow encountered in exp
  Y = 1.0 / (1.0 + np.exp(-Y))
/usr/local/lib/python3.4/dist-packages/sklearn/linear_model/sag.py:267: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Logistic regression trained. 7.9e+02s.
Test batch #0
Test batch #1
Test batch #2
Test batch #3
Test batch #4
Test batch #5
Test batch #6
Test batch #7
Test batch #8
Test batch #9
Test batch #10
Test batch #11
Test batch #12
Test batch #13
Test batch #14
Test batch #15
Random forest score: 0.4241
Logistic regression score: 0.4756


/home/timgaripov/work/prac/semestr5/05/autoencoder.py:119: RuntimeWarning: overflow encountered in exp
  Y = 1.0 / (1.0 + np.exp(-Y))


In [ ]:
print('s')

In [35]:
run_classifiers(lambda data: autoencoder_features(data, theta, hidden_size, visible_size, step=4))

Train fearutes generated. 38s.
RF trained. 2.6e+02s.


/home/timgaripov/work/prac/semestr5/05/autoencoder.py:119: RuntimeWarning: overflow encountered in exp
  Y = 1.0 / (1.0 + np.exp(-Y))
/usr/local/lib/python3.4/dist-packages/sklearn/linear_model/sag.py:267: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Logistic regression trained. 3e+03s.
Test batch #0
Test batch #1
Test batch #2
Test batch #3
Test batch #4
Test batch #5
Test batch #6
Test batch #7
Test batch #8
Test batch #9
Test batch #10
Test batch #11
Test batch #12
Test batch #13
Test batch #14
Test batch #15
Random forest score: 0.4343
Logistic regression score: 0.5058


/home/timgaripov/work/prac/semestr5/05/autoencoder.py:119: RuntimeWarning: overflow encountered in exp
  Y = 1.0 / (1.0 + np.exp(-Y))


In [43]:
hidden_size = np.array([100, 64, 100])
visible_size = 192
rho = 0.05
lambda_ = 1e-6
beta = 0.5
loss = lambda theta: autoencoder_loss(theta, visible_size, hidden_size, lambda_, rho, beta, patches)
theta = initialize(hidden_size, visible_size)
opt_res = scipy.optimize.minimize(loss, theta, method='L-BFGS-B', jac=True, options={'disp': True,
                                                                                      'maxiter': 2000
                                                                                    })
theta = opt_res['x']
np.savez('theta_3.npz', theta=theta)

In [45]:
run_classifiers(lambda data: autoencoder_features(data, theta, hidden_size, visible_size, step=8))

Train fearutes generated. 18s.
RF trained. 6.5e+02s.


/home/timgaripov/work/prac/semestr5/05/autoencoder.py:119: RuntimeWarning: overflow encountered in exp
  Y = 1.0 / (1.0 + np.exp(-Y))
/usr/local/lib/python3.4/dist-packages/sklearn/linear_model/sag.py:267: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Logistic regression trained. 6.4e+02s.
Test batch #0
Test batch #1
Test batch #2
Test batch #3
Test batch #4
Test batch #5
Test batch #6
Test batch #7
Test batch #8
Test batch #9
Test batch #10
Test batch #11
Test batch #12
Test batch #13
Test batch #14
Test batch #15
Random forest score: 0.4002
Logistic regression score: 0.4128


/home/timgaripov/work/prac/semestr5/05/autoencoder.py:119: RuntimeWarning: overflow encountered in exp
  Y = 1.0 / (1.0 + np.exp(-Y))


In [44]:
W, b = get_params(theta, visible_size, hidden_size)
display_layer(W[0].T, 'check.png')